## 설정

In [13]:
# 파이썬 2와 파이썬 3 지원
from __future__ import division, print_function, unicode_literals

# 공통
import numpy as np
import os
import tensorflow as tf
from functools import partial

learning_rate = 0.01


# 일관된 출력을 위해 유사난수 초기화
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# 맷플롯립 설정
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
plt.rcParams['font.family'] = 'HCR Batang'
plt.rcParams['axes.unicode_minus'] = False

# 그림을 저장할 폴더
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)
    
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

## 11.3 고속 옵티마이저
<br>

훈련 속도를 높일 수 있는 또 다른 방법으로 더 빠른 옵티마이저를 사용할 수 있습니다.
<br><br>
이 절에서는 모멘텀 최적화(Momentum optimization), 네스테로프 가속 경사(Nesterov Accelerated Gradient), AdaGard, RMSprop, Adam 옵티마이저를 소개하겠습니다.
<br><br>

### 11.3.1 모멘텀 최적화 (Momentum Optimization)
<br>

종단 속도에 도달할때 까지 빠르게 가속을 하겠다는 아이디어로 만들어 졌습니다. 일반적인 경사 하강법의 공식은 θ ←θ −η∇<sub>θ</sub> J (θ)입니다. 전 그래디언트는 고려하지 않고, 특정한 때의 그래디언트에 따라 속도가 아주 느려질수도 빨라질 수도 있습니다.

모멘텀 최적화의 공식은 
1. m ← βm+η∇<sub>θ</sub> J (θ)
2. θ ←θ − m
입니다. 
<br>

(m : 모멘텀 벡터 (Momentum vector), β : 모멘텀(Momentum, 하이퍼파라미터, 0과 1사이로 설정, 일반적으로 0.9))
<br><br>

모멘텀 최적화는 평편한 지역을 벗어날 수 있게 해주고, 골짜기의 최적점에 도달할때까지 점점 더 빠르게 내려갑니다. 또한 지역 최적점(Local optima)를 건너뛰는데도 도움이 됩니다. 
<br><br>
단점이라고 하면 하이퍼파라미터가 하나 늘어나는 것입니다. 그러나 보통 0.9에서 잘 작동하며 경사 하강법보다 거의 항상 빠릅니다.

In [3]:
# 구현
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum = 0.9)

### 11.3.2 네스테로프 가속 경사 Nesterov  Accelerated Graident(NAG)
<br>

네스테로프 모멘텀 최적화(Nestero momentum optimization)라고도 불리우는 이 최적화는 1983년 유리 네스테로프(Yurii Nesterov)가 제한한 모멘텀 최적화의 한 변종입니다. 기존 모멘텀 최적화 보다 거의 항상 빠릅니다.
<br><br>
식은
1. m ← βm+η∇<sub>θ</sub> J (θ − βm)
2. θ ←θ − m
<br>

기존 모멘텀 최적화와 다른점은 그래디언트를 현재위치가 아니라 θ − βm에서 그래디언트를 측정한다는 것입니다. 모멘텀 벡터가 대부분 올바른 방향을 향할 것이기 때문에 조금더 나아가서 측정을 한다는 것이 아이디어이고, 그 덕분에 훈련 속도가 더 빨라졌습니다.

In [4]:
# 구현
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9, use_nesterov=True)

![모멘텀최적화들](./image/1.png)

###  11.3.3 AdaGrad
<br>

한쪽이 길쭉한 그릇이 있다고 생각해보죠. 경사 하강법은 먼저 가장 가파른 경사로 내려가서 골짜기에 도달해 천천히 아래로 이동합니다. AadGrad알고리즘은 이런 약간의 비효율적인 부분을 해결합니다. 바로 목적지로 향하는 것이죠.
<br><br>
식은 이러합니다.
![AdaGrad 식](./image/2.png)
<br>
(첫번째 식의 특수기호는 원소별 곱셈을 나타내고, 두번째 식의 특수기호는 원소별 나눗셈을 나타냅니다.)
<br><br>

매 학습률이 $\sqrt{s_i + ε}$ 로 나눠집니다. 즉, 경사가 가파를 때는 학습률이 낮아지고, 경사가 완만할때는 학습률이 높아지는 효과를 가져다줍니다. 이를 적응적 학습률(Adaptive learning rate)라고 합니다. 이는 전역 최적점 방향으로 곧장 가게하는데 도움을 줍니다. 학습률 하이퍼파라메터를 덜 튜닝해도 되는 것또한 이 최적화의 장점입니다.
<br><br>
![AdaGrad그래프](./image/3.png)
<br><br>

AdaGrad는 간단한 2차방정식에는 잘 작동하지만 신경망을 훈련시킬때는 일찍 멈추는 경향이 있습니다. 따라서 심층 신경망에는 가급적 사용을 피하는 것이 좋습니다.

In [5]:
# 구현
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)

### 11.3.4 RMSProp
<br>

RMSProp는 빠르게 느려져서 전역 최적점에 도달하지 못하는 AdaGrad의 문제점을 해결한 최적화방법입니다. 가장 최근 반복에서 비롯된 그래디언트만 누적함으로써 해결을 했고 이를 위해 첫번째 단계에서 지수 감소를 사용합니다.
<br><br>

![RMSProp 식](./image/4.png)
<br>
(β : 보통 감쇠율, 보통 0.9로 설정, 이 기본값이 잘 작동하므로 따로 튜닝할 필요가 없음)
<br><br>

간단한 문제를 제외하고 이 최적화가 언제나 AdaGrad보다 성능이 좋습니다. Adam최적화가 나오기 전까지 연구자들이 가장 선호하는 최적화 알고리즘 이었습니다.

In [6]:
# 구현
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate, momentum=0.9, decay=0.9, epsilon=1e-10)

### 11.3.5 Adam 최적화(Adaptive Moment Estimation)
<br>

모멘텀 최적화와 RMSProp의 아이디어를 합친것입니다.
<br><br>

식은
![Adam최적화 식](./image/5.png)
<br>
(T는 (1부터 시작하는) 반복횟수를 나타냅니다.)
<br><br>

1,2,5는 모멘텀 최저과, RMSProp와 흡사합니다. m와 s는 0으로 초기화 되기때문에 초기에 0쪽으로 치우치게 됩니다. 3,4단계는 초기에 이 두값을 증폭시키는데 도움을 줍니다.
<br>
$\beta_1$은 보통 0.9로 초기화 하고 $\beta_2$는 0.999로 초기화 합니다. ε은 보통 아주작은수로 초기화합니다.
<br>
적응적 학습률 알고리즘이기때문에 학습률 하이퍼파라미터를 튜닝할 필요는 없습니다. 

In [7]:
# 구현
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

여기서 소개된 최적화들은 1차 편미분(Jacobian)에만 의존합니다. 최적화 이론에는 2차 편미분(Hessian)을 기반으로한 뛰어난 알고리즘이 있습니다. 하지만 심층신경망에 쓰기에는 계산량이 너무 많습니다.

### 11.3.6 학습률 스케줄링
<br>

적응적 학습률 알고리즘이 아닌이상 최적의 학습률을 찾는것은 쉽지 않습니다.
<br><br>

![학습률 스케줄링](./image/6.png)
<br><br>

일정한 학습률보다 더 좋은 방법은 학습 중에 유동적으로 학습률을 적용시키는 것입니다. 이것을 학습 스케줄이라고 합니다. 가장 보편적인 것은 다음과 같습니다.
<br><br>
- 미리 정의된 개별적인 고정 학습률
예를 들어 처음에 η<sub>0</sub>= 0.1로 학습률을 지정하고 50 에포크 후에 η<sub>1</sub>= 0.001로 바꿉니다. 이 방법이 잘 작동
할 수는 있지만 적절한 학습률과 적당한 시점을 찾으려면 이리저리 바꿔봐야 합니다.
- 성능 기반 스케줄링
매 N 스텝마다 (조기 종료처럼) 검증 오차를 측정하고 오차가 줄어들지 않으면 λ만큼 학습률을 감소시킵니다.
- 지수 기반 스케줄링
반복 횟수 t의 함수 $η(t)=η_010^{-t/r}$ 로 학습률을 설정합니다. 이 방법이 잘 작동하지만 $η_0$와 r을 튜닝해야
합니다. 학습률은 매 r 스텝마다 1/10씩 줄어들 것입니다.
- 거듭제곱 기반 스케줄링
학습률을 $η(t)=η_0(1+t/r)^{-c}$ 으로 설정합니다. 하이퍼파라미터 c는 보통 1로 지정됩니다. 지수 기반 스케
줄링과 비슷하지만 학습률이 훨씬 느리게 감소됩니다.

In [8]:
# 구현
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

# 학습률 스케쥴링
with tf.name_scope("train"):       # 책에는 없음
    initial_learning_rate = 0.1
    decay_steps = 10000
    decay_rate = 1/10
    global_step = tf.Variable(0, trainable=False, name="global_step")
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step,
                                               decay_steps, decay_rate)
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss, global_step=global_step)
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 5
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "검증 세트 정확도:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 검증 세트 정확도: 0.9574
1 검증 세트 정확도: 0.9716
2 검증 세트 정확도: 0.973
3 검증 세트 정확도: 0.9798
4 검증 세트 정확도: 0.9816


## 11.4 과대적합을 피하기 위한 규제 방법
<br>

심층 신경망에는 수만개, 수백만개의 파라미터가 있습니다. 때문에 네트워크 자유도가 매우 높으며 크고 복잡한 데이터셋을 학습시킬 수 있습니다. 하지만 이는 과대적합의 위험성도 내포하고있습니다.
<br><br>
이번에는 이 과대적합을 피할 수 있는 규제방법들(조기종료, $ℓ_1과 ℓ_2$규제, 트롭아웃, 맥스-노름 규제, 데이터 증식 등)을 소개하고 텐서플로우로 구현해보는 시간을 가지겠습니다.
<br><br>

### 11.4.1 조기종료
<br>

검증 세트로 모델을 평가해서 이전의 최고 성능보다 더 나을 경우 이를 저장하고 이 마지막 저장 이후 스탭을 카운터해서 어떤 한계점(예를들어 2000스탭)을 넘으면 훈련을 중지시킵니다. 그리고 마지막으로 저장된 것을 불러옵니다.
<br><br>
이는 이 자체로도 잘 작동되지만, 다른 규제 기법과 연결하면 더 높은 성능을 얻을 수 있습니다.
<br><br>

### 11.4.2 $ℓ_1과 ℓ_2$규제
<br>

$ℓ_1과 ℓ_2$를 사용해 신경망의 연결 가중치에 제약을 줄 수있습니다. (편향에는 적용하지 않는다고 해요.)

In [10]:
# 구현
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    logits = tf.layers.dense(hidden1, n_outputs, name="outputs")
    
W1 = tf.get_default_graph().get_tensor_by_name("hidden1/kernel:0")
W2 = tf.get_default_graph().get_tensor_by_name("outputs/kernel:0")

scale = 0.001 # l1 규제 하이퍼파라미터

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")
    reg_losses = tf.reduce_sum(tf.abs(W1)) + tf.reduce_sum(tf.abs(W2))
    loss = tf.add(base_loss, scale * reg_losses, name="loss")
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 20
batch_size = 200

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "검증 세트 정확도:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 검증 세트 정확도: 0.831
1 검증 세트 정확도: 0.871
2 검증 세트 정확도: 0.8838
3 검증 세트 정확도: 0.8934
4 검증 세트 정확도: 0.8966
5 검증 세트 정확도: 0.8988
6 검증 세트 정확도: 0.9016
7 검증 세트 정확도: 0.9044
8 검증 세트 정확도: 0.9058
9 검증 세트 정확도: 0.906
10 검증 세트 정확도: 0.9068
11 검증 세트 정확도: 0.9054
12 검증 세트 정확도: 0.907
13 검증 세트 정확도: 0.9084
14 검증 세트 정확도: 0.9088
15 검증 세트 정확도: 0.9064
16 검증 세트 정확도: 0.9068
17 검증 세트 정확도: 0.9066
18 검증 세트 정확도: 0.9066
19 검증 세트 정확도: 0.9052


하지만 이런 방식은 층이 많으면 번거롭습니다. 텐서플로우에는 더 좋은 방법이 있습니다. 변수를 생성하는 함수(get_variable()이나 tf.layers.dense()같은 함수)는 각 변수에 대한 \*\_regularizer 매개변수를 제공합니다. (예를들면 kernel_regularizer).

In [14]:
# 구현

reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

scale = 0.001

my_dense_layer = partial(
    tf.layers.dense, activation=tf.nn.relu,
    kernel_regularizer=tf.contrib.layers.l1_regularizer(scale))

with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    hidden2 = my_dense_layer(hidden1, n_hidden2, name="hidden2")
    logits = my_dense_layer(hidden2, n_outputs, activation=None,
                            name="outputs")
    
with tf.name_scope("loss"):                                     # 책에는 없음
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(  # 책에는 없음
        labels=y, logits=logits)                                # 책에는 없음
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")   # 책에는 없음
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = tf.add_n([base_loss] + reg_losses, name="loss")
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 20
batch_size = 200

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "검증 세트 정확도:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 검증 세트 정확도: 0.8274
1 검증 세트 정확도: 0.8766
2 검증 세트 정확도: 0.8952
3 검증 세트 정확도: 0.9016
4 검증 세트 정확도: 0.908
5 검증 세트 정확도: 0.9096
6 검증 세트 정확도: 0.9124
7 검증 세트 정확도: 0.9154
8 검증 세트 정확도: 0.9178
9 검증 세트 정확도: 0.919
10 검증 세트 정확도: 0.92
11 검증 세트 정확도: 0.9224
12 검증 세트 정확도: 0.9212
13 검증 세트 정확도: 0.9228
14 검증 세트 정확도: 0.9222
15 검증 세트 정확도: 0.9218
16 검증 세트 정확도: 0.9218
17 검증 세트 정확도: 0.9228
18 검증 세트 정확도: 0.9216
19 검증 세트 정확도: 0.9214


### 11.4.3 드롭 아웃
<br>

심층신경망에서 가장 잘 쓰는 규제방법입니다. 2012년 제프리 힌튼이 제안한 방법입니다.
<br><br>
알고리즘은 간단합니다. 매 훈련 스탭에서 각 뉴런은(입출력 뉴런 제외) 임시적으로 드롭아웃될 확률 p를 가집니다. 즉, 매 훈련마다 제외될 수 도있고 아닐 수도 있습니다. p는 하이퍼파라미터로 드롭아웃 비율(Dropout rate)라고 하며 보통 50%로 설정됩니다.
<br><br>

![드롭아웃](./image/7.png)
<br><br>

하나 기술적인 세부사항이 있습니다. p를 50%로 두면 한 뉴런이 훈련때보다 평균적으로 두배 이상의 입력뉴런과 연결이 됩니다. 이 점을 보상하기 위해 훈련하고 나서 각 뉴런의 연결 가중치에 0.5를 곱할 필요가 있습니다. 좀 더 일반화 해서 말하자면 각 연결 가중치에 보존 확율(Keep probability) (1-p)를 곱해야 합니다. 
<br><br>
배치 정규화에서 했던것 처럼 훈련할떄는 training 매개변수를 True로, 테스트할때는 False로 지정해야 합니다.
<br><br>
모델이 과대적합하면 드롭아웃 비율을 올리고, 과소적합하면 드롭아웃 비율을 낮춰야합니다.

In [15]:
# 구현
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

training = tf.placeholder_with_default(False, shape=(), name='training')

dropout_rate = 0.5  # == 1 - keep_prob
X_drop = tf.layers.dropout(X, dropout_rate, training=training)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X_drop, n_hidden1, activation=tf.nn.relu,
                              name="hidden1")
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)
    hidden2 = tf.layers.dense(hidden1_drop, n_hidden2, activation=tf.nn.relu,
                              name="hidden2")
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training)
    logits = tf.layers.dense(hidden2_drop, n_outputs, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 20
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={training: True, X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "검증 세트 정확도:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 검증 세트 정확도: 0.9258
1 검증 세트 정확도: 0.945
2 검증 세트 정확도: 0.9488
3 검증 세트 정확도: 0.9576
4 검증 세트 정확도: 0.9612
5 검증 세트 정확도: 0.9618
6 검증 세트 정확도: 0.962
7 검증 세트 정확도: 0.9668
8 검증 세트 정확도: 0.968
9 검증 세트 정확도: 0.97
10 검증 세트 정확도: 0.9692
11 검증 세트 정확도: 0.9698
12 검증 세트 정확도: 0.9712
13 검증 세트 정확도: 0.9718
14 검증 세트 정확도: 0.9712
15 검증 세트 정확도: 0.9692
16 검증 세트 정확도: 0.971
17 검증 세트 정확도: 0.9732
18 검증 세트 정확도: 0.9724
19 검증 세트 정확도: 0.9726


### 11.4.4 맥스 노름 규제 (Max-Norm Regularization)
<br>

신경망에서 아주 널리 사용되는 규제방법입니다.
<br><br>
이 방식은 각각 뉴런에 대해 입력의 연결가중치 w가 w ≤$\lVert r\rVert_2$ 이 되도록 제한합니다. r은 맥스-노름 하이퍼파라미터이고 $\lVert · \rVert_2$는 $ℓ_2$노름을 나타냅니다.
<br><br>
일반적으로 $\lVert r\rVert_2$를 계산한 다음 w를 클리핑($w \leftarrow w \frac{r}{\lVert r\rVert_2}$)합니다. r을 줄이면 규제 정도가 커져 과대적합을 감소시킵니다. 맥스-노름 규제는 (배치 정규화를 사용하지 않았을때) 그래디언트 감소/폭주 문제를 완화하는데도 도움을 줍니다.
<br><br>
텐서 플로우에 내장되어있지는 않지만 구현하기 쉽습니다.

In [16]:
# 구현
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

learning_rate = 0.01
momentum = 0.9

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# clip_by_norm()함수를 사용해 가중치를 클리핑 하는 함수를 만듬
threshold = 1.0
weights = tf.get_default_graph().get_tensor_by_name("hidden1/kernel:0")
clipped_weights = tf.clip_by_norm(weights, clip_norm=threshold, axes=1)
clip_weights = tf.assign(weights, clipped_weights)

# 같은 방식으로 두번째층
weights2 = tf.get_default_graph().get_tensor_by_name("hidden2/kernel:0")
clipped_weights2 = tf.clip_by_norm(weights2, clip_norm=threshold, axes=1)
clip_weights2 = tf.assign(weights2, clipped_weights2)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 20
batch_size = 50

with tf.Session() as sess:                                              # 책에는 없음
    init.run()                                                          # 책에는 없음
    for epoch in range(n_epochs):                                       # 책에는 없음
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):  # 책에는 없음
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            clip_weights.eval()
            clip_weights2.eval()                                        # 책에는 없음
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid}) # 책에는 없음
        print(epoch, "검증 세트 정확도:", accuracy_val)                     # 책에는 없음

    save_path = saver.save(sess, "./my_model_final.ckpt")               # 책에는 없음

0 검증 세트 정확도: 0.9568
1 검증 세트 정확도: 0.97
2 검증 세트 정확도: 0.9708
3 검증 세트 정확도: 0.9774
4 검증 세트 정확도: 0.977
5 검증 세트 정확도: 0.9786
6 검증 세트 정확도: 0.982
7 검증 세트 정확도: 0.9822
8 검증 세트 정확도: 0.9802
9 검증 세트 정확도: 0.9818
10 검증 세트 정확도: 0.9828
11 검증 세트 정확도: 0.9846
12 검증 세트 정확도: 0.9828
13 검증 세트 정확도: 0.9838
14 검증 세트 정확도: 0.9844
15 검증 세트 정확도: 0.9848
16 검증 세트 정확도: 0.9842
17 검증 세트 정확도: 0.984
18 검증 세트 정확도: 0.9848
19 검증 세트 정확도: 0.985


In [17]:
# 더 깔끔하게하려면 max_norm_regularizer() 함수를 만드는 것.
def max_norm_regularizer(threshold, axes=1, name="max_norm",
                         collection="max_norm"):
    def max_norm(weights):
        clipped = tf.clip_by_norm(weights, clip_norm=threshold, axes=axes)
        clip_weights = tf.assign(weights, clipped, name=name)
        tf.add_to_collection(collection, clip_weights)
        return None # 규제 손실을 위한 항이 없습니다
    return max_norm

reset_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

learning_rate = 0.01
momentum = 0.9

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

max_norm_reg = max_norm_regularizer(threshold=1.0)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu,
                              kernel_regularizer=max_norm_reg, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu,
                              kernel_regularizer=max_norm_reg, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 20
batch_size = 50

clip_all_weights = tf.get_collection("max_norm")

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            sess.run(clip_all_weights)
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid}) # 책에는 없음
        print(epoch, "검증 세트 정확도:", accuracy_val)                      # 책에는 없음

    save_path = saver.save(sess, "./my_model_final.ckpt")                # 책에는 없음

0 검증 세트 정확도: 0.9556
1 검증 세트 정확도: 0.9706
2 검증 세트 정확도: 0.9686
3 검증 세트 정확도: 0.9736
4 검증 세트 정확도: 0.9768
5 검증 세트 정확도: 0.9754
6 검증 세트 정확도: 0.9802
7 검증 세트 정확도: 0.9808
8 검증 세트 정확도: 0.9838
9 검증 세트 정확도: 0.982
10 검증 세트 정확도: 0.9812
11 검증 세트 정확도: 0.9828
12 검증 세트 정확도: 0.9828
13 검증 세트 정확도: 0.9828
14 검증 세트 정확도: 0.9832
15 검증 세트 정확도: 0.9842
16 검증 세트 정확도: 0.9832
17 검증 세트 정확도: 0.9828
18 검증 세트 정확도: 0.9828
19 검증 세트 정확도: 0.9836


### 데이터 증식(Data Augmentation)
<br>

데이터 증식은 말그대로 기존의 데이터에서 새로운 데이터를 만들어 데이터의 크기를 임의로 늘립니다. 이상적으로는 새로운 데이터들도 기조의 데이터와 유사해야 합니다. 기존 데이터와 연관성의 거의 없는 단순한 백색소음(White noise)를 추가하는것은 도움이 되지 않습니다.
<br><br>
Mnist 데이터 셋의 숫자 7을 예로 들어보겠습니다. 여기에 데이터 증식을 한다고 하면 추가되는 데이터는 7을 기울이거나 다양한 각도로 기울이거나 크기를 바꾼 것들이 됩니다. 하지만 전혀 엉뚱한(9 ,3과 같은 다른 숫자) 데이터는 학습에 전혀 도움이 되지 않습니다.

## 11.5 실용적 가이드라인
<br>

기본적으로 쓰이는 DNN 설정은 아래와 같습니다.
<br><br>
![실용적 가이드라인](./image/8.png)
<br><br>

만약 비슷한 문제를 해결한 모델을 찾았다면 그 신경망의 일부를 사용하는 것이 효율적입니다.
<br><br>

다음과 같은 경우 위 설정을 바꿀 필요가 있습니다.
<br>

- 좋은 학습률을 찾을 수 없다면 (수렴이 너무 느릴 때 학습 속도를 올리면 수렴은 빨라지지만 네트워크의 정확도는 최적화가 덜 되므로) 지수 감소 같은 학습 스케줄을 추가해볼 수 있습니다.
- 훈련 세트가 너무 작다면 데이터 증식을 수행할 수 있습니다.
- 희소 모델이 필요하면 $ℓ_1$ 규제를 추가합니다(또는 훈련이 끝난 뒤 작은 가중치를 0으로 만듭니다). 만약 더욱 희박한 희소 모델이 필요하면 네스테로프 가속 경사나 Adam 옵티마이저 대신 $ℓ_1$ 규제와 함께 FTRL 알고리즘을 사용할 수 있습니다.
- 실행 속도가 아주 빠른 모델을 필요로 하면 배치 정규화를 빼고 ELU 활성화 함수를 LeakyReLU로 바꿉니다. 희소 모델을 만드는 것도 도움이 됩니다.